# All Techniques Of Hyper Parameter Optimization
1) GridSearchCV
2) RandomizedSearchCV
3) Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
4) Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)
5) Optuna- Automate Hyperparameter Tuning
6) Genetic Algorithms (TPOT Classifier)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv(r'C:\Users\user\Pictures\PYTHON_PANDAS\diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [4]:
#### Independent And Dependent features
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [5]:
print("shape of X : ",X.shape)
print("shape of y : ",y.shape)

shape of X :  (768, 8)
shape of y :  (768,)


### Train Test Split

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=33)

In [11]:
print("shape of X_train : ",X_train.shape)
print("shape of y_train : ",y_train.shape)
print("shape of X_test : ",X_test.shape)
print("shape of y_test : ",y_test.shape)

shape of X_train :  (614, 8)
shape of y_train :  (614,)
shape of X_test :  (154, 8)
shape of y_test :  (154,)


In [12]:
y.value_counts()


0    500
1    268
Name: Outcome, dtype: int64

-- Data is not imbalanced

## RandomForestCLasssifier 

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("------------------------------------------------------------------")
print(accuracy_score(y_test,prediction))
print("------------------------------------------------------------------")
print(classification_report(y_test,prediction))

[[87 12]
 [24 31]]
------------------------------------------------------------------
0.7662337662337663
------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.78      0.88      0.83        99
           1       0.72      0.56      0.63        55

    accuracy                           0.77       154
   macro avg       0.75      0.72      0.73       154
weighted avg       0.76      0.77      0.76       154



### The main parameters used by a Random Forest Classifier are:

--> criterion = the function used to evaluate the quality of a split. || 
--> max_depth = maximum number of levels allowed in each tree.  ||
--> max_features = maximum number of features considered when splitting a node.  ||
--> min_samples_leaf = minimum number of samples which can be stored in a tree leaf.   ||
--> min_samples_split = minimum number of samples necessary in a node to cause node splitting.   ||
--> n_estimators = number of trees in the ensamble.   ||

In [24]:
### Manual Hyperparameter Tuning
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[87 12]
 [28 27]]
0.7402597402597403
              precision    recall  f1-score   support

           0       0.76      0.88      0.81        99
           1       0.69      0.49      0.57        55

    accuracy                           0.74       154
   macro avg       0.72      0.68      0.69       154
weighted avg       0.73      0.74      0.73       154



# Randomized Search Cv


In [25]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [1,3,4,5,7,9]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]

random_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features ,
    'max_depth' : max_depth ,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'criterion' : ['entropy','gini']
}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [27]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator= rf, param_distributions= random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1 )
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [29]:
rf_randomcv.best_score_

0.7769249163079867

In [30]:
best_random_grid=rf_randomcv.best_estimator_


In [32]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print()
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print()
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[87 12]
 [24 31]]

Accuracy Score 0.7662337662337663

Classification report:               precision    recall  f1-score   support

           0       0.78      0.88      0.83        99
           1       0.72      0.56      0.63        55

    accuracy                           0.77       154
   macro avg       0.75      0.72      0.73       154
weighted avg       0.76      0.77      0.76       154



## GridSearch CV

In [33]:
rf_randomcv.best_params_

{'n_estimators': 1000,
 'min_samples_split': 7,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 890,
 'criterion': 'gini'}

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 500, rf_randomcv.best_params_['n_estimators'] - 300, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 200, rf_randomcv.best_params_['n_estimators'] + 300]
}

print(param_grid)

{'criterion': ['gini'], 'max_depth': [890], 'max_features': ['auto'], 'min_samples_leaf': [2, 4, 6], 'min_samples_split': [5, 6, 7, 8, 9], 'n_estimators': [500, 700, 1000, 1200, 1300]}


In [36]:
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  7.0min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [890],
                         'max_features': ['auto'],
                         'min_samples_leaf': [2, 4, 6],
                         'min_samples_split': [5, 6, 7, 8, 9],
                         'n_estimators': [500, 700, 1000, 1200, 1300]},
             verbose=2)

In [37]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=890, min_samples_leaf=6, min_samples_split=6,
                       n_estimators=1300)

In [38]:
best_grid=grid_search.best_estimator_
best_grid

RandomForestClassifier(max_depth=890, min_samples_leaf=6, min_samples_split=6,
                       n_estimators=1300)

In [39]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print()
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print()
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[87 12]
 [27 28]]

Accuracy Score 0.7467532467532467

Classification report:               precision    recall  f1-score   support

           0       0.76      0.88      0.82        99
           1       0.70      0.51      0.59        55

    accuracy                           0.75       154
   macro avg       0.73      0.69      0.70       154
weighted avg       0.74      0.75      0.74       154



# Automated Hyperparameter Tuning
Automated Hyperparameter Tuning can be done by using techniques such as

i) Bayesian Optimization               ||
ii) Gradient Descent                     ||
iii) Evolutionary Algorithms

# Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

1) Objective Function = defines the loss function to minimize.
2) Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
3) Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [41]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [42]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }

In [43]:
space


{'criterion': <hyperopt.pyll.base.Apply at 0x1f5362398d0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x1f536239780>,
 'max_features': <hyperopt.pyll.base.Apply at 0x1f536239668>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x1f5362390b8>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x1f536239048>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x1f536249128>}

In [44]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [45]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|██████████| 80/80 [13:38<00:00, 10.24s/trial, best loss: -0.7736771957883514]


{'criterion': 0,
 'max_depth': 900.0,
 'max_features': 3,
 'min_samples_leaf': 0.06631012438389859,
 'min_samples_split': 0.2355488991647348,
 'n_estimators': 5}

In [46]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

entropy
None
1300


In [47]:

trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[89 10]
 [28 27]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.76      0.90      0.82        99
           1       0.73      0.49      0.59        55

    accuracy                           0.75       154
   macro avg       0.75      0.69      0.71       154
weighted avg       0.75      0.75      0.74       154



# Genetic Algorithms
Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's imagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [48]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [49]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.7769501740089975

Generation 2 - Current best internal CV score: 0.7818096935743994

Generation 3 - Current best internal CV score: 0.7818096935743994

Generation 4 - Current best internal CV score: 0.7818096935743994

Generation 5 - Current best internal CV score: 0.7818096935743994

Best pipeline: RandomForestClassifier(input_matrix, criterion=gini, max_depth=230, max_features=log2, min_samples_leaf=8, min_samples_split=14, n_estimators=2000)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [51]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.7467532467532467


# Optimize hyperparameters of the model using Optuna
The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [52]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [53]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-01-26 12:11:09,894] A new study created in memory with name: no-name-62f3dbd3-3c25-4c70-ab64-52d03c49f039
[I 2021-01-26 12:11:34,853] Trial 0 finished with value: 0.7736649131197195 and parameters: {'classifier': 'RandomForest', 'n_estimators': 470, 'max_depth': 23.923494668706127}. Best is trial 0 with value: 0.7736649131197195.
[I 2021-01-26 12:11:36,058] Trial 1 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 0.03061584728961105}. Best is trial 0 with value: 0.7736649131197195.
[I 2021-01-26 12:11:36,993] Trial 2 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 0.015753216001108145}. Best is trial 0 with value: 0.7736649131197195.
[I 2021-01-26 12:11:37,042] Trial 3 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 9.023103589809458e-05}. Best is trial 0 with value: 0.7736649131197195.
[I 2021-01-26 12:11:40,034] Trial 4 finished with value: 0.772030926191615 and

[I 2021-01-26 12:12:32,428] Trial 35 finished with value: 0.767136936075243 and parameters: {'classifier': 'RandomForest', 'n_estimators': 530, 'max_depth': 17.94149607255196}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:12:35,441] Trial 36 finished with value: 0.772038896859557 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1100, 'max_depth': 26.301336746293043}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:12:37,613] Trial 37 finished with value: 0.7720229555236728 and parameters: {'classifier': 'RandomForest', 'n_estimators': 940, 'max_depth': 36.06563119148975}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:12:38,643] Trial 38 finished with value: 0.7622748286306393 and parameters: {'classifier': 'RandomForest', 'n_estimators': 390, 'max_depth': 23.625918770768557}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:12:40,564] Trial 39 finished with value: 0.7671289654073011 and parameters

[I 2021-01-26 12:13:32,618] Trial 70 finished with value: 0.7687629523354057 and parameters: {'classifier': 'RandomForest', 'n_estimators': 680, 'max_depth': 33.91002077252934}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:13:35,153] Trial 71 finished with value: 0.7655109198150805 and parameters: {'classifier': 'RandomForest', 'n_estimators': 860, 'max_depth': 29.67245522663525}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:13:36,478] Trial 72 finished with value: 0.7638928742228598 and parameters: {'classifier': 'RandomForest', 'n_estimators': 580, 'max_depth': 71.17195571107239}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:13:37,645] Trial 73 finished with value: 0.7622588872947552 and parameters: {'classifier': 'RandomForest', 'n_estimators': 450, 'max_depth': 81.53136145180552}. Best is trial 33 with value: 0.7801928901641958.
[I 2021-01-26 12:13:38,423] Trial 74 finished with value: 0.7704128805993943 and parameters:

Accuracy: 0.7801928901641958
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 730, 'max_depth': 25.90196922553536}


In [54]:
trial

FrozenTrial(number=33, values=[0.7801928901641958], datetime_start=datetime.datetime(2021, 1, 26, 12, 12, 27, 369795), datetime_complete=datetime.datetime(2021, 1, 26, 12, 12, 29, 352212), params={'classifier': 'RandomForest', 'n_estimators': 730, 'max_depth': 25.90196922553536}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100, low=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=33, state=TrialState.COMPLETE, value=None)

In [55]:
study.best_params


{'classifier': 'RandomForest',
 'n_estimators': 730,
 'max_depth': 25.90196922553536}

In [56]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=30, n_estimators=330)

In [57]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[86 13]
 [23 32]]
0.7662337662337663
              precision    recall  f1-score   support

           0       0.79      0.87      0.83        99
           1       0.71      0.58      0.64        55

    accuracy                           0.77       154
   macro avg       0.75      0.73      0.73       154
weighted avg       0.76      0.77      0.76       154

